In [ ]:
import datetime as dt
import dateutil.rrule as rr
import importlib as ilib
import logging
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
import pandas.tseries.offsets as o
import plotly.offline as plo
import plotly.graph_objs as go

import omega.configuration as oc
import omega.core.chain as occ
import omega.core.instrument as oci

plo.init_notebook_mode(connected=True)
#%matplotlib inline

In [ ]:
# Disable logging output
log = logging.getLogger()
log.setLevel(logging.ERROR)

In [ ]:
stem = 'FC'
# Get Chain
fc = occ.FutureChain(stem, oci.FutureType.Outright)
contracts = fc.initialize_contracts(occ.Status.All)
data = fc.initialize_data(days_back=240, partial=True)

In [ ]:
def get_value(df, day):
    try:
        return df[df.index <= day].iloc[-1]['Close']
    except:
        return None

In [ ]:
start = dt.datetime.strptime('2011-01-01','%Y-%m-%d')
first_friday = start + o.BDay(4 - start.weekday())
ycs = dict()
# chain below is the dataframe of all contracts
for date in rr.rrule(rr.WEEKLY, dtstart=first_friday, until=dt.datetime.strptime('2017-12-01','%Y-%m-%d')):
    day = date.strftime('%Y-%m-%d')
    actives = fc.chain[fc.chain['LastDate'] > day]
    ts = [actives.iloc[idx]['Ticker'] for idx in range(4)]
    ycs[day] = [get_value(fc.data[x], day) for x in ts]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
import matplotlib.pyplot as plt
import numpy as np

zs = []
fig = plt.figure()
ax = fig.gca(projection='3d')
verts = []

for w in ycs:
    xs, ys = [1, 2, 3, 4, 5, 6, 7], ycs[w]
    verts.append(list(zip(xs, ys)))
zs = range(len(ycs))

colours = plt.cm.Blues(np.linspace(0.2, 1, len(zs)))
poly = PolyCollection(verts, facecolors=colours)
ax.add_collection3d(poly, zs=zs, zdir='y')

ax.set_xlabel('X')
ax.set_xlim3d(1, 7)
ax.set_ylabel('Y')
ax.set_ylim3d(1, len(ycs) + 1)
ax.set_zlabel('Z')
ax.set_zlim3d(200, 500)
plt.show()

In [ ]:
length = 6
days = ['2017-03-23', '2017-04-23', '2017-05-23', '2017-06-23', '2017-07-23', '2017-08-23', '2017-09-23', '2017-10-23', '2017-11-23', '2017-12-23', '2018-01-23', '2018-02-23', '2018-03-23']
index = range(length)

values = []
i = [x + 1 for x in index]
for d in days:
    actives = fc.chain[fc.chain['LastDate'] > d]
    print('Day: {} - Tickers: {}'.format(d, actives['Ticker'].values))
    log.warning('')
    ts = [actives.iloc[idx]['Ticker'] if (idx + 1) <= len(actives) else None for idx in index]
    values.append([get_value(fc.data[x], d) if x is not None else None for x in ts])

plo.iplot({
    'data': [go.Scatter(x=i, y=v, name=days[i]) for i,v in enumerate(values)],
    'layout': go.Layout(title='Forward Curve', xaxis=dict(title='Contracts'), yaxis=dict(type='log', title='Price'))
}, show_link=False)